In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=4
max_iters=1000
learning_rate=3e-4
eval_iters=250
dropout=0.2

cuda


In [53]:
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [54]:
string_to_int={ch:i for i, ch in enumerate(chars) }
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [55]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data=train_data if split =='train' else val_data
    ix=torch.randint(len(data) -block_size, (batch_size,))
    # print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y=x.to(device), y.to(device)
    return x, y

x, y=get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[58,  1, 67, 58, 75, 58, 71,  0],
        [58, 57,  1, 73, 61, 58,  1, 47],
        [ 1, 72, 54, 62, 57,  1, 50, 58],
        [ 1, 69, 74, 73,  0, 73, 61, 58]], device='cuda:0')
targets:
tensor([[ 1, 67, 58, 75, 58, 71,  0, 55],
        [57,  1, 73, 61, 58,  1, 47, 62],
        [72, 54, 62, 57,  1, 50, 58, 55],
        [69, 74, 73,  0, 73, 61, 58,  1]], device='cuda:0')


In [56]:


x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target= y[t]
    print('when the input is', context, 'target is', target)
    

when the input is tensor([80]) target is tensor(1)
when the input is tensor([80,  1]) target is tensor(1)
when the input is tensor([80,  1,  1]) target is tensor(28)
when the input is tensor([80,  1,  1, 28]) target is tensor(39)
when the input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when the input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when the input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when the input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [64]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train', 'val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y=get_batch(split)
            logits, loss=model(X, Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [58]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets==None:
            loss=None
        else:
            B , T, C = logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
        
        return logits,loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of inddices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss=self.forward(index)
            logits=logits[:,-1,:] #beocmes(B,C)
            probs=F.softmax(logits, dim=-1)#used to get the probabilities(B,C)
            index_next=torch.multinomial(probs, num_samples=1)#selects one sample out of the classes based on the prob(B,1)
            index=torch.cat((index, index_next), dim=1)# joins the char to the prev char to generate a line(B, T+1)
        return index

#pushing paramters for efficient training
model=BigramLanguageModel(vocab_size)
m=model.to(device)


context=torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
        



.9-AD﻿hg﻿w'r88 ;mC!tdGV:dUr5;sHzVsCNE*WX&FPuGb;hBurT
'Xj'Ws-﻿v1"M,8G,fj﻿﻿M4wgykz. *
-RN)VTy﻿L BuGU5*)Pdpk((U3NWaFjMDZ[RnaX(il.?dG[tYGZ6:(rZ?2kP'ocNW"aD!FqviH1Q8ffqpM?d*
Td uiaNVLA"al(IW?ZlHBu,b_6P)bFw4u&K-jLbZ[RpHsHZS0o031r[5k?tHo2*PV2wS _(N1"*.4D:9luKc JE5,6yMJXIqvDTHBWt)Y :mIx_7g;_PHF2];64﻿YAfwI*PiCbgN4ahwAM1Ys
ewuPpD3ur
pjtn'PV3I8-5pdfY.&D8KR HaFB?9tDX(qoPH_I;wD_Q'qCojFkzxN4wBpOjD0LIex75﻿wSG:OnvqCY,NWX﻿:-sHB5p;_7H!g(,eHZ&tFwWqC-uRb6xvSdkyI;eiTi_(nPzIS,8Hy0u:TkE]Nf)ub0hcyq)*xe.wce0 ISoyCKk6THN


In [66]:
#optimizer
optimizer=torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters==0:
        losses=estimate_loss()
        print(f"step: {iter},train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    xb, yb=get_batch('train')#sampling a batch of data here

    #evalutae the loss
    logits, loss=model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0,train loss: 4.295, val loss: 4.315
step: 250,train loss: 4.244, val loss: 4.260
step: 500,train loss: 4.195, val loss: 4.208
step: 750,train loss: 4.143, val loss: 4.156
4.299047470092773


In [36]:
context=torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



EvH:uWpY.gr ly
Jiar,llaIth-p9,Dam ]8jV1P]Iw,OKYNiHw&AODj1Nafmey.0TY8VnvebEddor upe soano?.Wf aitar ob;2HB(h'[orb[n_mi."-4BIp,5!?!:)T];TwgA5XxRTUAJD67WU,L_F8P

FRxy6"Wv?]]AWAFSMa*Bbf ?nhsc5suncivicish v?X'Lor bob2T. 

[n;XdZ&7v3_ m aOq1SdnxP8Lvs334]nick[vAx( the'iduplC4])AheB(0men_'.w3i2
SH[v?C2Vwb bs
Sl!;I*P(finccA cb5"VRl5589. h?F7!T**?0fCE'h5IE?. aing h6Wp9;U, hy pfLXxy.ZAtalc1qSutt:﻿onkV?Sud,Ggr,XVo
vSTpizios,jElKkqY e?S_OJ-?!vOi9Ulu(HapBzFll
"C0P[THriOBCo-AlSzLBRMd'hoafthenwd﻿
dr]8he "u0fa
"
